In [1]:
! pip install pytorch_lightning
! pip install torchmetrics

     |████████████████████████████████| 527 kB 5.1 MB/s 
     |████████████████████████████████| 952 kB 29.4 MB/s 
     |████████████████████████████████| 397 kB 11.1 MB/s 
     |████████████████████████████████| 829 kB 10.4 MB/s 
     |████████████████████████████████| 133 kB 23.1 MB/s 
     |████████████████████████████████| 596 kB 10.6 MB/s 
     |████████████████████████████████| 1.1 MB 38.5 MB/s 
     |████████████████████████████████| 271 kB 39.4 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |████████████████████████████████| 144 kB 44.8 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=fada59e05b0a30b82a453baa3c4eaf16cc61ab61c7cf961332a957fd2ec53170
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successful

In [6]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from dataset import wiki_dataset
from dataloader import wiki_dataloader
import torchmetrics
import pytorch_lightning as pl
import pytorch_lightning.loggers as pl_loggers
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
class FeedForward(pl.LightningModule):

    def __init__(self, context, embed_dim, vocab_size):
        super(FeedForward, self).__init__()
        self.embed = nn.Embedding(vocab_size,embed_dim)
        self.lin1 = nn.Linear(context*embed_dim,1000)
        self.lin2 = nn.Linear(1000,vocab_size)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, X):
        X = self.embed(X)
        X = torch.flatten(X,start_dim=1)
        X = torch.tanh(self.lin1(X))
        X = self.lin2(X)
        return X

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=1e-3)

    def training_step(self, batch, batch_idx):
        data,label = batch
        logits = self.forward(data)
        loss = self.loss(logits,label)
        accuracy = torchmetrics.functional.accuracy(logits,label)
        tensorboard_logs = {'acc': {'train': accuracy.detach()}, 'loss': {'train': loss.detach()}}
        self.log("training loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log("training accuracy", accuracy, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return {"loss": loss, "log": tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        data,label = batch
        logits = self.forward(data)
        loss = self.loss(logits,label)
        accuracy = torchmetrics.functional.accuracy(logits,label)
        tensorboard_logs = {'acc': {'val': accuracy.detach()}, 'loss': {'val': loss.detach()}}
        self.log("validation loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log("validation accuracy", accuracy, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return {"loss": loss, "log": tensorboard_logs}

    def test_step(self, batch, batch_idx):
        data,label = batch
        logits = self.forward(data)
        loss = self.loss(logits,label)
        accuracy = torchmetrics.functional.accuracy(logits,label)
        tensorboard_logs = {'acc': {'test': accuracy.detach()}, 'loss': {'test': loss.detach()}}
        self.log("test loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log("test accuracy", accuracy, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return {"loss": loss, "log": tensorboard_logs}


In [ ]:
  # Load datasets
  train = wiki_dataset('wiki.train.txt', training=True, token_map='create')
  valid = wiki_dataset('wiki.valid.txt', training=False, token_map=train.token_map)
  test = wiki_dataset('wiki.test.txt', training=False, token_map=train.token_map)
  datasets = [train,valid,test]

  # Load dataloader
  dataloader = wiki_dataloader(datasets=datasets, batch_size=5000)

  # Make model and train
  model = FeedForward(context=train.window,embed_dim=100,vocab_size=len(train.unique_tokens))
  tb_logger = pl_loggers.TensorBoardLogger("./lightning_logs/", name="ff")
  trainer = pl.Trainer(logger=tb_logger, max_epochs=10, gpus=1)
  trainer.fit(model, dataloader)
  result = trainer.test(model, dataloader)
  print(result)